<a href="https://colab.research.google.com/github/AkhileshPandeyji/Udacity-deep-gocolab/blob/master/ArtificialNeuralNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import random

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.ERROR)


1.14.0


In [0]:
# initializations

BATCH_SIZE = 100
n_inputs = 784
# inputs for input layer

#X = tf.placeholder(tf.float32,[None,n_inputs])
X = tf.placeholder(tf.float32,[None,28,28,1])
pkeep = tf.placeholder(tf.float32)

# flattened vector
X = tf.reshape(X,[-1,784])
 
# known labels
Y_ = tf.placeholder(tf.float32,[None,10])


In [0]:
# weights and biases
# Five fully-connected layers

K = 512
L = 256
M = 128
N = 64
out = 10

W1 = tf.Variable(tf.truncated_normal([784,K],stddev=0.1))
B1 = tf.Variable(tf.zeros([K]))

W2 = tf.Variable(tf.truncated_normal([K,L],stddev=0.1))
B2 = tf.Variable(tf.zeros([L]))

W3 = tf.Variable(tf.truncated_normal([L,M],stddev=0.1))
B3 = tf.Variable(tf.zeros([M]))

W4 = tf.Variable(tf.truncated_normal([M,N],stddev=0.1))
B4 = tf.Variable(tf.zeros([N]))

W5 = tf.Variable(tf.truncated_normal([N,out],stddev=0.1))
B5 = tf.Variable(tf.zeros([out]))

In [0]:
# Model Creation using relu and softmax - can also use sigmoid instead of relu

Y1 = tf.nn.relu(tf.matmul(X,W1)+B1)
Y2 = tf.nn.relu(tf.matmul(Y1,W2)+B2)
Y3 = tf.nn.relu(tf.matmul(Y2,W3)+B3)
Y4 = tf.nn.relu(tf.matmul(Y3,W4)+B4)
Y5 = tf.nn.dropout(Y4,pkeep)
Ylogits = tf.matmul(Y5,W5)+B5
Y = tf.nn.softmax(Ylogits)

In [0]:
# Success Metrics - Loss, Optimizer and accuracy
cross_entropy = -tf.reduce_sum(Y_*tf.log(Y))
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_,logits=Ylogits))
optimizer = tf.train.AdamOptimizer(0.0003)
train_step = optimizer.minimize(cross_entropy)

is_correct = tf.equal(tf.argmax(Y_,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [0]:
# Running Session and training model
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)



In [7]:
# importing libraries for getting mnist dataset
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

num_train_examples = mnist.train.num_examples
num_val_examples = mnist.validation.num_examples
num_test_examples = mnist.test.num_examples


print(num_train_examples,num_val_examples,num_test_examples)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 5000 10000


In [8]:
for epoch in range(1000):
#   print("Epoch-{}/1000".format(epoch))
  batch_x,batch_y = mnist.train.next_batch(BATCH_SIZE)
  train_dict = {X:batch_x,Y_:batch_y,pkeep:0.75}
  sess.run(train_step,feed_dict=train_dict)
  
#   #training accuracy and loss
#   t_a,t_c = sess.run([accuracy,cross_entropy],feed_dict=train_dict)
  
#   #Validation accuracy and loss
#   v_a,v_c = sess.run([accuracy,cross_entropy],feed_dict={X:mnist.test.images,Y_:mnist.test.labels,pkeep:1.0})
  
#   print("Training: Accuracy:{},Loss:{} ".format(t_a,t_c))
#   print("Validation: Accuracy:{},Loss:{}".format(v_a,v_c))
  
  if epoch % 100 == 0:
    a,c = sess.run([accuracy,cross_entropy],feed_dict={X:batch_x,Y_:batch_y,pkeep:1.0})
    print("Training: Accuracy:{},Loss:{} ".format(a,c))
    v_a,v_c = sess.run([accuracy,cross_entropy],feed_dict={X:mnist.test.images,Y_:mnist.test.labels,pkeep:1.0})
    print("Validation: Accuracy:{},Loss:{}".format(v_a,v_c))

Training: Accuracy:0.23000000417232513,Loss:215.1061553955078 
Validation: Accuracy:0.1256999969482422,Loss:22718.51953125
Training: Accuracy:0.8899999856948853,Loss:39.52033233642578 
Validation: Accuracy:0.8949999809265137,Loss:3741.42626953125
Training: Accuracy:0.9300000071525574,Loss:23.112367630004883 
Validation: Accuracy:0.9262999892234802,Loss:2557.6396484375
Training: Accuracy:0.9300000071525574,Loss:22.470884323120117 
Validation: Accuracy:0.9361000061035156,Loss:2200.409423828125
Training: Accuracy:0.9399999976158142,Loss:18.946699142456055 
Validation: Accuracy:0.9460999965667725,Loss:1812.1640625
Training: Accuracy:0.9200000166893005,Loss:30.562145233154297 
Validation: Accuracy:0.9490000009536743,Loss:1629.0869140625
Training: Accuracy:0.9900000095367432,Loss:6.955602645874023 
Validation: Accuracy:0.9544000029563904,Loss:1493.4208984375
Training: Accuracy:0.9800000190734863,Loss:13.240516662597656 
Validation: Accuracy:0.9581999778747559,Loss:1396.489501953125
Training:

In [9]:
import PIL.Image as Image

img_file = tf.keras.utils.get_file("test2digit.png",origin='https://raw.githubusercontent.com/do-community/tensorflow-digit-recognition/master/test_img.png')
test_img = np.invert(Image.open(img_file).convert('L')).ravel()
print(test_img.shape)
prediction = sess.run(tf.argmax(Y,1),feed_dict={X:[test_img],pkeep:1.0})
print(prediction[0])

(784,)
2
